In [ ]:
# Import key librarys
import pandas as pd
import numpy as np
import os
import glob
import re

In [ ]:
# Find details of input file
def find_details(f):

    # Get filename after / character
    filename = f.split('/')[-1]
    
    # Look for numbers in filename
    num = re.findall(r'\d+', filename)
    
    for n in num:
        
        # If number more than 3 characters assume it is a year 
        if len(n) > 3:
           
            # Set year to n
            year = n
    
    # Look at filename before ' 2' and ' CCR' and assume is org
    s = filename.split(' 2', 1)[0]
    s = s.split(' - Climate Change Duties annual report', 1)[0]
    s = s.split(' CCR', 1)[0]
    
    # Set org to s
    org = s
    
    return org, year

# Tidy up strings
def tidy(df):
   
    # Remove parenthesis
    df = df.applymap(lambda x: x.replace('(', '') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(')', '') if type(x) is str else x)

    # Strip whitespace
    df = df.applymap(lambda x: x.strip() if type(x) is str else x)
    
    # Make all uppercase
    df = df.applymap(lambda x: x.upper() if type(x) is str else x)
    # Make columns all uppercase
    df.columns = map(lambda x: str(x).upper(), df.columns)
    
    # Replace annoying substrings
    df = df.applymap(lambda x: x.replace(' And ', ' & ') if type(x) is str else x)
    # Below stops df.to_csv reading text as a command to create a new column!
    df = df.applymap(lambda x: x.replace('&AMP;', '&') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace('&;', '&') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace('  ', ' ') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(' – ', ' - ') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(' / ', '/') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace('/ ', '/') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace(' /', '/') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace('KGCO2E', 'KG CO2E') if type(x) is str else x)
    df = df.applymap(lambda x: x.replace('TCO2E', 'T CO2E') if type(x) is str else x)
    
    # Replace Blank values with NaN
    df = df.replace(r'^\s*$', np.nan, regex=True)
    
    # Remove any line breaks
    df = df.replace(r'\n',' ', regex = True)

    return df

# Replace empty cells with np.nan
def drop(df):
    
    df = df.replace("", np.nan) 
    df = df.dropna(how='all', axis=1)
    
    return df
    
# Function to convert into kWh
def convert_energy(row):
    
    # Check if consumption units column has the value KWH
    if row['consumption_units'] == 'KWH':

        # Return same value
        return row['consumption']

    # Check if consumption units column has the value LITRES
    if row['consumption_units'] == 'LITRES':
        
        # Return value multiplied by 10
        return row['consumption'] * 10

    # Transport sources
    tsources1 = ['RAIL', 'BUS', 'AIR', 'FERRY','TAXI']

    # Transport Energy Use Figures from Sustainable Energy Without Hot Air p141
    # Car - 68 kWh per 100 passenger-km
    # Rail - 6 kWh per 100 passenger-km
    # Bus - 19 kWh per 100 passenger-km
    # Air - 51 kWh per 100 passenger-km
    # Sea - 57 kWh per 100 passenger-km
    #  Overall transport efficiencies of transport modes in Japan (1999).

    # Create diction with kWh per 100 passenger-km
    energykm = {'CAR':68, 'RAIL':6, 'BUS':19, 'AIR':51, 'FERRY': 57, 'LORRY':68, 'VAN':68, 'TAXI':68}

    # If a transport source
    if row['emission_source'] in tsources1:

        # If known units convert into 100 passenger-km, then kWh
        if row['consumption_units'] == 'PASSENGER KM':

            # Calculate number of 100 passenger-km
            pkm100 = row['consumption'] / 100

        # Output kWh
        return energykm[row['emission_source']] * pkm100

    # If units are miles
    if row['consumption_units'] == 'MILES':

        # Calculate number of km
        km =  row['consumption'] / 1.6

        # Calculate number of litres (assume 8 litres per 100 km)
        litres =  km * 8 / 100

        return litres * 10

    # If units are miles
    if row['consumption_units'] == 'KM':

        # Calculate number of litres (assume 8 litres per 100 km)
        litres =  row['consumption'] * 8 / 100

        return litres * 10

    else:
        # Return NaN
        return np.nan

In [ ]:
# use glob to get all the csv files in the folder
path = './highland/'
xlsx_files = glob.glob(os.path.join(path, "*.xlsx"))

# Create list for all dfs
frames = []

for f in xlsx_files:
    
    print(f)
    
    # Get list of sheet names
    sheet_names = pd.ExcelFile(f).sheet_names
    
    # Store organisation and year in variables
    org, year = find_details(f)
    
    # Check type of template
    if len(sheet_names) > 5:
        # Define sheet number
        nsheet = 3
    else: 
        nsheet = 0
        
    # Read in sheet with emissions data
    # Red warnings due to data validation on spreadsheet
    df = pd.read_excel(f, sheet_name = sheet_names[nsheet], engine='openpyxl')
    
    # Tidy
    df = tidy(df)
    
    # Drop
    df = drop(df)
    
    # Look for full match of header in df and return index
    header = 'EMISSION SOURCE'
    index = df[df.apply(lambda row: row.astype(str).str.fullmatch(header).any(), axis = 1)].index[0]
    
    # Drop dodgy rows at the top of sheet
    df = df.iloc[index:].reset_index()
    
    # Grab the first row for the header
    new_header = df.iloc[0] 
    # Take the data less the header row
    df = df[1:]
    # Set the header row as the df header
    df.columns = new_header
    
    # Find location of useful columns
    a = df.columns.get_loc('EMISSION SOURCE')
    b = df.columns.get_loc('SCOPE')
    c = df.columns.get_loc('CONSUMPTION DATA')
    array = df.columns.get_loc('UNITS')
    e = df.columns.get_loc('EMISSION FACTOR')
    g = df.columns.get_loc('EMISSIONS T CO2E')
    h = df.columns.get_loc('COMMENTS')
    
    # Select important columns
    df = df.iloc[:, [a, b, c, np.where(array == True)[0][0], e, np.where(array == True)[0][1], g, h]]
    
    # Look for NaN values in Emission source column
    null_index = df.loc[pd.isna(df["EMISSION SOURCE"]), :].index
    
    # Create a list
    ls = []
    
    # Loop through 5 values to make a list of step sizes in null_index
    for i in range(5):
        ls.append(null_index[i + 1] - null_index[i])
        
    # Define cut off
    cut_off = null_index[0 + ls.index(1)] - 1

    # Drop All Rows After First Occurrence of two NaNs in a row
    df = df[:cut_off]
    
    # Add Columns with year and org
    # Subtract 1 from year as 2020 report is for 2019 etc.
    df['YEAR'] = int(year) - 1
    df['ORGANISATION'] = org
    
    frames.append(df)

In [ ]:
#df.tail()

In [ ]:
# Concat frames together
dft = pd.concat(frames)

# Drop rows with less than 6 valid data cells
dft = dft.dropna(thresh = 6)

# Manually rename headings
dft.columns = ['emission_source', 'scope', 'consumption', 'consumption_units', 'emission_factor', 
                         'emission_factor_units', 'emissions_TCO2e', 'comments', 'year', 'organisation']

In [ ]:
# Tidy up some names of emission sources
di = {'GRID ELECTRICITY GENERATION': 'ELECTRICITY GENERATION',
      'GRID ELECTRICITY TRANSMISSION': 'ELECTRICITY TRANSMISSION',
      'BURNING OIL KEROSENE': 'KEROSENE',
      'BIOMASS WOOD': 'BIOMASS WOOD',
      'GAS OIL': 'GAS OIL',
      'LPG': 'LPG',
      'DIESEL': 'DIESEL',
      'PETROL': 'PETROL',
      'CAR': 'CAR', # This ignores hybrid category
      'BUS': 'BUS',
      'RAIL': 'RAIL',
      'FERRY': 'FERRY',
      'TAXI': 'TAXI',
      'FLIGHTS': 'AIR',
      'HOMEWORKING EMISSIONS': 'HOMEWORKING',
      'LANDFILL': 'LANDFILL',
      'GARDEN WASTE COMPOSTING': 'GARDEN COMPOSTING',
      'FOOD & DRINK COMPOSTING': 'FOOD & DRINK COMPOSTING',
     }

# Function to help with renaming emission sources
def rename(old, new):
    dft.emission_source = dft.emission_source.apply(lambda x: new if old in x else x)

for d in di:
    rename(d, di[d])
    
dft.consumption_units = dft.consumption_units.apply(lambda x: '% FTES' 
                                                    if 'PERCENTAGE OF TOTAL FTES' in x else x) 

In [ ]:
# Convert most units into kWh and place values into new column
dft['consumption_kWh'] = dft.apply(lambda row: convert_energy(row), axis=1)

In [ ]:
dft.to_csv('ssn_energy_highland.csv')

In [ ]:
dft.head(10)